<a href="https://colab.research.google.com/github/SoniaRFdz/TFG/blob/main/TFG2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

# Cargar los datos desde el archivo CSV
data = pd.read_csv('/content/simulated_pipe_flow_data.csv')

# Ver los primeros 5 registros para asegurarnos de que los datos se cargaron correctamente
print(data.head())


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Dividir los datos en X (características) e y (flujo)
X = data[['Diameter (m)', 'Pressure (Pa)', 'Velocity (m/s)']].values
y = data['Flow (m³/s)'].values

# Dividir los datos en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalizar las características
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)



import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Crear el modelo de red neuronal
model = Sequential()

# Capa de entrada y primera capa oculta con 64 neuronas
model.add(Dense(64, input_dim=3, activation='relu'))

# Segunda capa oculta con 32 neuronas
model.add(Dense(32, activation='relu'))

# Capa de salida para predecir un solo valor (flujo)
model.add(Dense(1))

# Compilar el modelo
model.compile(optimizer='adam', loss='mean_squared_error')

# Entrenar el modelo
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)


# Evaluar el modelo en los datos de prueba
loss = model.evaluate(X_test, y_test)
print(f"Error cuadrático medio en el conjunto de prueba: {loss}")

# Hacer predicciones
predicciones = model.predict(X_test)

# Imprimir algunas predicciones junto a los valores reales
for i in range(5):
    print(f"Predicción: {predicciones[i][0]}, Valor real: {y_test[i]}")

#FIRST ATTEMPT: FAIL. Cuadratic error too elevate. Possible solutions: 1)More layers or neurons, 2)not enough training.


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam

# Cargar los datos desde el archivo CSV
data = pd.read_csv('simulated_pipe_flow_data.csv')

# Dividir los datos en X (características) e y (flujo)
X = data[['Diameter (m)', 'Pressure (Pa)', 'Velocity (m/s)']].values
y = data['Flow (m³/s)'].values

# Normalizar la salida (flujo) para que esté en la misma escala que las características
scaler_y = StandardScaler()
y_normalized = scaler_y.fit_transform(y.reshape(-1, 1))

# Dividir en datos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y_normalized, test_size=0.2, random_state=42)

# Normalizar las características
scaler_X = StandardScaler()
X_train = scaler_X.fit_transform(X_train)
X_test = scaler_X.transform(X_test)

# Crear el modelo de red neuronal
model = Sequential()

# Añadir la capa de entrada explícita
model.add(Input(shape=(3,)))

# Primera capa oculta con regularización L2
model.add(Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)))

# Segunda capa oculta
model.add(Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)))

# Tercera capa oculta con dropout
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.3))

# Capa de salida
model.add(Dense(1))

# Compilar el modelo con una tasa de aprendizaje ajustada
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='mean_squared_error')

# Entrenar el modelo
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2, verbose=0)

# Evaluar el modelo
loss = model.evaluate(X_test, y_test)
print(f"Error cuadrático medio en el conjunto de prueba: {loss}")

# Hacer predicciones
predicciones = model.predict(X_test)

# Mostrar algunas predicciones
for i in range(5):
    print(f"Predicción: {predicciones[i][0]}, Valor real: {y_test[i][0]}")

    #SECOND ATTEMPT: SUCCESS, but improvable


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0189  
Error cuadrático medio en el conjunto de prueba: 0.01920049637556076
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
Predicción: -0.5777406692504883, Valor real: -0.600146044159068
Predicción: 0.13815996050834656, Valor real: 0.2354100300526558
Predicción: -0.20154434442520142, Valor real: -0.2713056669496903
Predicción: -0.3667179346084595, Valor real: -0.42104820605960974
Predicción: -0.461667001247406, Valor real: -0.5002226862459505
